In [1]:
#!pip install langchain-community==0.2.4 langchain==0.2.3 faiss-cpu==1.8.0 unstructured==0.14.5 unstructured[pdf]==0.14.5 transformers==4.41.2 sentence-transformers==3.0.1

**Importing the Dependencies**

In [1]:
import os

from langchain_community.llms import Ollama
from langchain.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA

from langchain_community.document_loaders import JSONLoader
import json
from pathlib import Path
from pprint import pprint


In [2]:
# loading the LLM
llm = Ollama(
    model="llama3.1",
    temperature=0
)

/tmp/ipykernel_190946/3762590127.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


In [6]:
# loading the document
loader = UnstructuredFileLoader("Manual d'instal·lació dels equips informàtics.pdf")
documents = loader.load()

In [3]:
loader = JSONLoader(
    file_path='./data.json',
    jq_schema='.',
    text_content=False)

documents = loader.load()

In [4]:
# create document chunks
text_splitter = CharacterTextSplitter(separator="/n",
                                      chunk_size=1000,
                                      chunk_overlap=200)

In [5]:
text_chunks = text_splitter.split_documents(documents)

In [6]:
# loading the vector embedding model
embeddings = HuggingFaceEmbeddings()

/tmp/ipykernel_190946/803604610.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipykernel_190946/803604610.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/home/aborregop2/Descargas/myenv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable

In [8]:
knowledge_base = FAISS.from_documents(text_chunks,embeddings)

In [9]:
# retrieval QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=knowledge_base.as_retriever())

In [ ]:
 #PDF
question = "De que trata este documento?"
response = qa_chain.invoke({"query": question})
print(response["result"])

In [10]:
 #JSON
question = "Que informacion me da este documento?"
response = qa_chain.invoke({"query": question})
print(response["result"])

ValueError: Ollama call failed with status code 500. Details: {"error":"model requires more system memory (5.9 GiB) than is available (5.3 GiB)"}

In [29]:
question = "Que hi ha de dinar el dilluns?"
response = qa_chain.invoke({"query": question})
print(response["result"])

Segons el menú del cafè, el dilluns hi ha opcions normals i variables. Les opcions normals són:

* Arròs a la cubana
* Bacallà al forn amb ceba i pastanaga

Les opcions variables són:

* Espaguetis al pesto
* Mandonguilles amb tomàquet i peusols
